In [1]:
import json
import pandas as pd

In [2]:
fieldsOfStudy = set()

In [3]:
with open('dblp_papers_v11.txt', 'r') as file:
    for line in file:
        data = json.loads(line)
        for fos in data.get('fos',[]):
            fieldsOfStudy.add(fos.get('name',''))

In [31]:
data.keys()

dict_keys(['id', 'title', 'authors', 'venue', 'year', 'n_citation', 'page_start', 'page_end', 'doc_type', 'publisher', 'volume', 'issue', 'references', 'indexed_abstract', 'fos'])

In [33]:
'indexed_abstract' in data.keys()

True

In [2]:
AITopLevelTopics = ['Artificial intelligence', 'Computer vision', 'Data mining',
                     'Data science', 'Machine learning', 'Natural language processing',
                     'Pattern recognition', 'Speech recognition']

In [8]:
for topic in AITopLevelTopics:
    if topic not in fieldsOfStudy:
        print('Not found topic: ', topic)

In [13]:
count = 0
with open('dblp_papers_v11.txt', 'r') as file:
    for line in file:
        if (count < 2):
            count += 1
            continue
        data = json.loads(line)
        break
            

In [14]:
data

{'id': '1000022707',
 'title': 'A SIMPLE OBSERVATION REGARDING ITERATIONS OF FINITE-VALUED POLYNOMIAL-TIME FUNCTIONS',
 'authors': [{'name': 'Jerzy Mycka', 'id': '263067851'}],
 'venue': {'raw': 'Reports on Mathematical Logic', 'id': '42464861'},
 'year': 2009,
 'n_citation': 0,
 'page_start': '19',
 'page_end': '29',
 'doc_type': 'Journal',
 'publisher': '',
 'volume': '44',
 'issue': '',
 'references': ['1972178849', '2069792094'],
 'indexed_abstract': {'IndexLength': 49,
  'InvertedIndex': {'A': [0],
   'b': [1],
   's': [2],
   't': [3],
   'r': [4],
   'a': [5, 31],
   'c': [6],
   't.': [7],
   'We': [8],
   'present': [9],
   'this': [10],
   'note': [11],
   'to': [12, 25],
   'point': [13],
   'out': [14],
   'that': [15],
   'the': [16, 43],
   'finitevalued': [17],
   'polynomial-time': [18],
   'computable': [19, 47],
   'functions': [20],
   'are': [21],
   'closed': [22],
   'with': [23],
   'respect': [24],
   'iteration.': [26],
   'This': [27],
   'fact': [28],
   'is'

In [17]:
abstract = [w for w in data['indexed_abstract']['InvertedIndex'].keys() if len(w) > 1]

In [19]:
AITopLevelTopicsSet = set(AITopLevelTopics)
papersUnderConsideration = set()
with open('dblp_papers_v11.txt', 'r') as file:
    for line in file:
        data = json.loads(line)
        for fos in data.get('fos',[]):
            if fos.get('name','') in AITopLevelTopicsSet:
                papersUnderConsideration.add(data['id'])

In [3]:
# paperList = list(papersUnderConsideration)
# with open("dblpPaperIDs.json", 'w') as f:
#     json.dump(paperList, f)
# To Read
with open("dblpPaperIDs.json", 'r') as f:
    paperList = json.load(f)
papersUnderConsideration = set(paperList)

In [18]:
paperData = dict()
with open('dblp_papers_v11.txt', 'r') as file:
    with open('dblp_AIpapers_v11.json', 'w') as outfile:
        for line in file:
            data = json.loads(line)
            paperID = data.get('id','')
            if paperID in papersUnderConsideration:
                dataDict = dict()
                dataDict['title'] = data.get('title', '')
                references = list()
                for reference in data.get('references',[]):
                    if reference in papersUnderConsideration:
                        references.append(reference)
                dataDict['references'] = references
                dataDict['abstract'] = []
                if 'indexed_abstract' in data:
                    dataDict['abstract'] = [w for w in data['indexed_abstract']['InvertedIndex'].keys() if len(w) > 1]                    
                dataDict['fos'] = data.get('fos',[])
                # paperData[paperID] = dataDict
                tmpDict = dataDict.copy()
                tmpDict['id'] = paperID
                json.dump(tmpDict, outfile)
                outfile.write('\n')

## Remove papers with citation count less than one

In [4]:
from collections import defaultdict
citationCount = defaultdict(int)
with open('dblp_AIpapers_v11.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        paperId = data['id']
        for paperCited in data.get('references', []):
            citationCount[paperCited] += 1
        

In [4]:
from collections import Counter

counter=Counter(list(citationCount.values()))


In [5]:
citationThreshold = 2
paperIdList = [key for key in citationCount.keys() if citationCount[key] > citationThreshold ]
IDoutfileName = "./data/dblpPaperIDs_" + str(citationThreshold) + "Thresholded.json"
with open(IDoutfileName, 'w') as f:
    json.dump(paperIdList, f)

In [6]:
papersUnderConsideration = set(paperIdList)
PapersOutFileName = './data/dblp_AIpapers' + str(citationThreshold) + 'Thresholded.json'
with open('dblp_AIpapers_v11.json', 'r') as file:
    with open(PapersOutFileName, 'w') as outfile:
        for line in file:
            data = json.loads(line)
            paperID = data.get('id','')
            if paperID in papersUnderConsideration:
                dataDict = dict()
                dataDict['title'] = data.get('title', '')
                references = list()
                for reference in data.get('references',[]):
                    if reference in papersUnderConsideration:
                        references.append(reference)
                dataDict['references'] = references
                dataDict['abstract'] = data['abstract']           
                dataDict['fos'] = data.get('fos',[])
                tmpDict = dataDict.copy()
                tmpDict['id'] = paperID
                json.dump(tmpDict, outfile)
                outfile.write('\n')

In [7]:
len(paperIdList)

475839

In [9]:
import os

filename = os.path.join('./data/tmp', 'lmao.txt')

In [10]:
filename

'./data/tmp/lmao.txt'